In [1]:
#This is used to mount your drive to the File
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
%cd /content/gdrive/My\ Drive/Coding_Test/K-NN

/content/gdrive/My Drive/Coding_Test/K-NN


In [0]:
#importing necessary libraries-numpy,os,pickle
import numpy as np
import os
import pickle


In [0]:
def loadData(path):
  #We create list to store the data
    listOfTestFiles = os.listdir(path=path)
    train = []
    train_labels = []
    test = []
    test_labels = []
    listOfTestFiles.sort()
    
    #For collecting Training data:
    #in the list created above the first five files are the training dataset, and the rest one file is test_set
    for file in listOfTestFiles[0:5]:
        with open(path+file,'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            train.append(dict[b'data'])
            train_labels.append(dict[b'labels'])
    
    
    #for collecting Testing data
    with open(path+listOfTestFiles[5],'rb') as fo:
            dict = pickle.load(fo,encoding='bytes')
            test.append(dict[b'data'])
            test_labels.append(dict[b'labels'])

    dictData = {}
    dictData['train_data'] = np.reshape(np.array(train),newshape=(np.array(train).shape[0]*np.array(train).shape[1],np.array(train).shape[2]))
    dictData['train_labels'] = np.reshape(np.array(train_labels),newshape=(np.array(train_labels).shape[0]*np.array(train_labels).shape[1]))
    dictData['test_data'] = np.reshape(np.array(test),newshape=(np.array(test).shape[0]*np.array(test).shape[1],np.array(test).shape[2]))
    dictData['test_labels'] = np.reshape(np.array(test_labels),newshape=(np.array(test_labels).shape[0]*np.array(test_labels).shape[1]))
    return dictData

In [0]:
#load the dataset
#Please enter the suitable location of the folder cifar-10-batches-py, downloaded from the site
dataset = loadData(path='/content/gdrive/My Drive/Coding_Test/K-NN/cifar-10-batches-py/')


In [0]:
x_train,y_train, x_test, y_test = dataset['train_data'],dataset['train_labels'],dataset['test_data'],dataset['test_labels']

In [9]:
#Splitting the data into train and validation set
# train = 49000 samples and validation set = 100 samples

train_x,train_y = x_train[0:49900],y_train[0:49900]
val_x, val_y = x_train[49900:],y_train[49900:]

print("No. of training samples = ",train_x.shape[0])
print("No. of validation set samples = ",val_x.shape[0])

No. of training samples =  49900
No. of validation set samples =  100


In [0]:
#Building the actual classifier:
#inputs will be of shape N X F where N = number of examples and F = number of features for each image.
#labels will be of size N,1. This consists of labels for all N labels from 10 classes
#This function uses the standard Euclidean Distance between the row pairs to compute the distance
class kNearestNeighbour(object):
    def __init__(self):
        pass
    
    def train(self,X,Y):
        #Knn will remember all of its training data
        self.Xtr = X
        self.Ytr = Y
        
    def predict(self,X,k):
        
        #to get no. of samples in train set 
        test_samples = X.shape[0]
        Ypred = np.zeros(test_samples,dtype=self.Ytr.dtype)
        
        for i in range(test_samples):
            
            print("Test example = ",i,end="\r")
            #label_count is array of zeros used to store the count for each class while comparing neighbours
            label_count = np.zeros(10,dtype=self.Ytr.dtype)   
            dist= np.array([])
            #This loop computes the cosine similarity between the rows of X and X_train for QUESTION 1.2
            for j in range(self.Xtr.shape[0]):
              cos_dis = np.dot(X[i,:],self.Xtr[j,:])/(np.linalg.norm(X[i,:])*np.linalg.norm(self.Xtr[j,:]))
              dist = np.append(dist,cos_dis)
            
            #idx will contain k smallest indices at the start of the list
            #this is called partial sorting
            #min_ind will help us to slice through k indices which have minimum distance
            idx = np.argpartition(dist,k)
            min_ind = idx[:k]
            
            #This for loop iterates over min_ind and find the label present in train-set_Y for x
            #and increase the count at that particular index whenever any label is repeated.
            #Max value at any index is returned using np.argmax() and stored at ith index of Ypred i.e. predicted values
            for x in min_ind:
                label_count[int(self.Ytr[x])] +=1
            Ypred[i] = np.argmax(label_count)
    
    
        return Ypred

In [0]:
knn = kNearestNeighbour()
knn.train(train_x,train_y)

In [0]:
#The is for Question 1.1
#This function returns the value of k using the highest occuring accuracy.
#This uses the assumption that the value of val accuracy increases to a maximum and then keeps reducing, and we return the value of K
def ValueOfk(val_x):
    
    val_accuracy = []
    val_accuracy.append(0)
    val_accuracy.append(0)
    k=2
    while True:
        val_y_pred = knn.predict(val_x,k)
        print(k)
        print(np.mean(val_y_pred==val_y))
        
        val_accuracy.append(np.mean(val_y_pred==val_y))
        
        if(val_accuracy[k]<val_accuracy[k-1]):
            break
        k=k+1
    return k-1
 

In [13]:
#This code takes more time compared to Euclidean Distance, so wait for few minutes for the code to run
#This happens as I am taking cosine similarity between each row pair in X_validate and X_train.
ValueofK= ValueOfk(val_x)
print(f"The Value of K for maximum accuracy is = {ValueofK}")

2
0.07
3
0.13
4
0.11
The Value of K for maximum accuracy is = 3
